![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/nlu/blob/master/examples/colab/component_examples/named_entity_recognition_NER/NLU_explain_clinical_doc_generic_pipeline.ipynb)

#Explain Clinical Document Generic

This pipeline is designed to:

- extract clinical/medical entities
- assign assertion status to the extracted entities
- establish relations between the extracted entities

from clinical texts. In this pipeline, 4 NER models, one assertion model, and one relation extraction model were used to achieve those tasks.

In [ ]:
! pip install nlu pyspark==3.1.2

In [ ]:
! pip install johnsnowlabs

In [1]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing NLU
! pip install --upgrade --q nlu --no-dependencies

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [2]:
import json
import os

import sparknlp
import sparknlp_jsl
import nlu

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 5.3.0
Spark NLP_JSL Version : 5.3.0


In [3]:
model = nlu.load("en.explain_doc.clinical_generic.pipeline")

explain_clinical_doc_generic download started this may take some time.
Approx size to download 1.7 GB
[OK!]


In [4]:
text = ["""Patient with severe fever and sore throat.
He shows no stomach pain. He maintained on an epidural and PCA for pain control.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer."""]

In [5]:
df = model.predict(text)

In [6]:
df

,assertion,document,entities_clinical_ner_chunk,entities_clinical_ner_chunk_class,entities_clinical_ner_chunk_confidence,entities_clinical_ner_chunk_origin_chunk,entities_clinical_ner_chunk_origin_sentence,entities_jsl_ner_chunk,entities_jsl_ner_chunk_class,entities_jsl_ner_chunk_confidence,...,relations_entity1_class,relations_entity1_end,relations_entity2,relations_entity2_begin,relations_entity2_class,relations_entity2_end,relations_origin_sentence,sentence_dl,unlabeled_dependency,word_embedding_embeddings
0,"[Present, Present, Absent, Past, Past, Hypothetical, Past, Present, Family]","Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzh...","[severe fever, sore throat, stomach pain, an epidural, PCA, pain control, CT, lung tumor, Alzheimer]","[PROBLEM, PROBLEM, PROBLEM, TREATMENT, TREATMENT, TREATMENT, TEST, PROBLEM, PROBLEM]","[0.902, 0.98815, 0.9933, 0.96599996, 0.9813, 0.73165, 0.9933, 0.9019, 0.9912]","[0, 1, 2, 3, 4, 5, 6, 7, 8]","[0, 0, 1, 2, 2, 2, 3, 3, 4]","[fever, sore throat, stomach pain, PCA, pain, CT, lung tumor, Alzheimer]","[TEST, PROBLEM, PROBLEM, TREATMENT, PROBLEM, TEST, PROBLEM, PROBLEM]","[0.9943, 0.69635, 0.82834995, 0.8087, 0.9875, 0.9934, 0.87944996, 0.9796]",...,PROBLEM,24,sore throat,30,PROBLEM,40,0,"[Patient with severe fever and sore throat., He shows no stomach pain., He maintained on an epidural and PCA for pain control., After CT, lung tumor located at the right lower lobe., Father with A...","[ROOT, fever, fever, Patient, sore, throat, fever, Patient, shows, ROOT, stomach, pain, shows, shows, maintained, ROOT, epidural, epidural, maintained, PCA, epidural, control, control, PCA, mainta...","[[-0.08453157544136047, 0.20632991194725037, -0.05394839495420456, -0.364096075296402, -0.7453896999359131, -0.45486748218536377, 0.1968030333518982, -0.7794908285140991, -0.08118873834609985, 0.2..."
0,"[Present, Present, Absent, Past, Past, Hypothetical, Past, Present, Family]","Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzh...","[severe fever, sore throat, stomach pain, an epidural, PCA, pain control, CT, lung tumor, Alzheimer]","[PROBLEM, PROBLEM, PROBLEM, TREATMENT, TREATMENT, TREATMENT, TEST, PROBLEM, PROBLEM]","[0.902, 0.98815, 0.9933, 0.96599996, 0.9813, 0.73165, 0.9933, 0.9019, 0.9912]","[0, 1, 2, 3, 4, 5, 6, 7, 8]","[0, 0, 1, 2, 2, 2, 3, 3, 4]","[fever, sore throat, stomach pain, PCA, pain, CT, lung tumor, Alzheimer]","[TEST, PROBLEM, PROBLEM, TREATMENT, PROBLEM, TEST, PROBLEM, PROBLEM]","[0.9943, 0.69635, 0.82834995, 0.8087, 0.9875, 0.9934, 0.87944996, 0.9796]",...,TREATMENT,96,PCA,102,TREATMENT,104,2,"[Patient with severe fever and sore throat., He shows no stomach pain., He maintained on an epidural and PCA for pain control., After CT, lung tumor located at the right lower lobe., Father with A...","[ROOT, fever, fever, Patient, sore, throat, fever, Patient, shows, ROOT, stomach, pain, shows, shows, maintained, ROOT, epidural, epidural, maintained, PCA, epidural, control, control, PCA, mainta...","[[-0.08453157544136047, 0.20632991194725037, -0.05394839495420456, -0.364096075296402, -0.7453896999359131, -0.45486748218536377, 0.1968030333518982, -0.7794908285140991, -0.08118873834609985, 0.2..."
0,"[Present, Present, Absent, Past, Past, Hypothetical, Past, Present, Family]","Patient with severe fever and sore throat.\nHe shows no stomach pain. He maintained on an epidural and PCA for pain control.\nAfter CT, lung tumor located at the right lower lobe. Father with Alzh...","[severe fever, sore throat, stomach pain, an epidural, PCA, pain control, CT, lung tumor, Alzheimer]","[PROBLEM, PROBLEM, PROBLEM, TREATMENT, TREATMENT, TREATMENT, TEST, PROBLEM, PROBLEM]","[0.902, 0.98815, 0.9933, 0.96599996, 0.9813, 0.7